In [ ]:
import nannyml as nml
from IPython.display import display

reference_df, analysis_df, analysis_target_df = nml.load_synthetic_car_loan_dataset()
analysis_full_df = analysis_df.merge(analysis_target_df, left_index=True, right_index=True)

column_names = [
    'car_value', 'salary_range', 'debt_to_income_ratio', 'loan_length', 'repaid_loan_on_prev_car', 'size_of_downpayment', 'driver_tenure', 'y_pred_proba', 'y_pred', 'repaid'
]

univ_calc = nml.UnivariateDriftCalculator(
    column_names=column_names,
    timestamp_column_name='timestamp',
    continuous_methods=['kolmogorov_smirnov', 'jensen_shannon'],
    categorical_methods=['chi2', 'jensen_shannon'],
    chunk_size=5000
)

univ_calc.fit(reference_df)
univariate_results = univ_calc.calculate(analysis_full_df)
display(univariate_results.filter(period='analysis', column_names=['debt_to_income_ratio']).to_df())

161 {'car_value': [<nannyml.drift.univariate.methods.KolmogorovSmirnovStatistic object at 0x7fee5ae62490>, <nannyml.drift.univariate.methods.JensenShannonDistance object at 0x7fee5ae627c0>], 'salary_range': [], 'debt_to_income_ratio': [<nannyml.drift.univariate.methods.KolmogorovSmirnovStatistic object at 0x7fee5ae62730>, <nannyml.drift.univariate.methods.JensenShannonDistance object at 0x7fee5ae62610>], 'loan_length': [<nannyml.drift.univariate.methods.KolmogorovSmirnovStatistic object at 0x7fee5ae62550>, <nannyml.drift.univariate.methods.JensenShannonDistance object at 0x7fee5ae62880>], 'repaid_loan_on_prev_car': [], 'size_of_downpayment': [], 'driver_tenure': [<nannyml.drift.univariate.methods.KolmogorovSmirnovStatistic object at 0x7fee5ae62760>, <nannyml.drift.univariate.methods.JensenShannonDistance object at 0x7fee5ae62910>], 'y_pred_proba': [<nannyml.drift.univariate.methods.KolmogorovSmirnovStatistic object at 0x7fee5ae625e0>, <nannyml.drift.univariate.methods.JensenShannonDist

chunk                                                             \
        chunk                                                              
  chunk_index                end_date end_index            key    period   
0           0 2018-11-30 00:27:16.848      4999       [0:4999]  analysis   
1           1 2018-12-30 07:03:16.848      9999    [5000:9999]  analysis   
2           2 2019-01-29 13:39:16.848     14999  [10000:14999]  analysis   
3           3 2019-02-28 20:15:16.848     19999  [15000:19999]  analysis   
4           4 2019-03-31 02:51:16.848     24999  [20000:24999]  analysis   
5           5 2019-04-30 09:27:16.848     29999  [25000:29999]  analysis   
6           6 2019-05-30 16:03:16.848     34999  [30000:34999]  analysis   
7           7 2019-06-29 22:39:16.848     39999  [35000:39999]  analysis   
8           8 2019-07-30 05:15:16.848     44999  [40000:44999]  analysis   
9           9 2019-08-29 11:51:16.848     49999  [45000:49999]  analysis   

                                  debt_to_income_ratio                  \
                                    kolmogorov_smirnov                   
           start_date start_index                alert lower_threshold   
0 2018-10-30 18:00:00           0                False            None   
1 2018-11-30 00:36:00        5000                False            None   
2 2018-12-30 07:12:00       10000                False            None   
3 2019-01-29 13:48:00       15000                False            None   
4 2019-02-28 20:24:00       20000                False            None   
5 2019-03-31 03:00:00       25000                False            None   
6 2019-04-30 09:36:00       30000                False            None   
7 2019-05-30 16:12:00       35000                False            None   
8 2019-06-29 22:48:00       40000                False            None   
9 2019-07-30 05:24:00       45000                False            None   

                                                                           \
                           jensen_shannon                                   
  upper_threshold    value          alert lower_threshold upper_threshold   
0            None  0.01576          False            None             0.1   
1            None  0.01268          False            None             0.1   
2            None  0.01734          False            None             0.1   
3            None  0.01280          False            None             0.1   
4            None  0.01918          False            None             0.1   
5            None  0.00824          False            None             0.1   
6            None  0.01058          False            None             0.1   
7            None  0.01002          False            None             0.1   
8            None  0.01068          False            None             0.1   
9            None  0.00680          False            None             0.1   

             
             
      value  
0  0.031661  
1  0.030011  
2  0.031129  
3  0.029464  
4  0.030809  
5  0.028681  
6  0.043628  
7  0.029253  
8  0.030628  
9  0.028330

In [ ]:
print(univariate_results.filter(period='analysis', column_names=['debt_to_income_ratio']).to_df().to_markdown(tablefmt="grid"))

+----+-------------------------------------+----------------------------------+-----------------------------------+-----------------------------+--------------------------------+------------------------------------+-------------------------------------+-----------------------------------------------------------+---------------------------------------------------------------------+---------------------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------+-----------------------------------------------------------------+-----------------------------------------------------------------+-------------------------------------------------------+
|    |   ('chunk', 'chunk', 'chunk_index') | ('chunk', 'chunk', 'end_date')   |   ('chunk', 'chunk', 'end_index') | ('chunk', 'chunk', 'key')   | ('chunk', 'chunk', 'period')   | ('chunk', 'chunk', 'start_date')   |   ('chunk', 'chunk', 'star

In [ ]:
alert_count_ranker = nml.AlertCountRanker()
alert_count_ranked_features = alert_count_ranker.rank(
    univariate_results.filter(methods=['jensen_shannon']),
    only_drifting = False)
display(alert_count_ranked_features)

,number_of_alerts,column_name,rank
0,5,y_pred_proba,1
1,5,salary_range,2
2,5,repaid_loan_on_prev_car,3
3,5,loan_length,4
4,5,car_value,5
5,0,y_pred,6
6,0,size_of_downpayment,7
7,0,repaid,8
8,0,driver_tenure,9
9,0,debt_to_income_ratio,10


In [ ]:
print(alert_count_ranked_features.to_markdown(tablefmt="grid"))

+----+--------------------+-------------------------+--------+
|    |   number_of_alerts | column_name             |   rank |
+====+====================+=========================+========+
|  0 |                  5 | y_pred_proba            |      1 |
+----+--------------------+-------------------------+--------+
|  1 |                  5 | salary_range            |      2 |
+----+--------------------+-------------------------+--------+
|  2 |                  5 | repaid_loan_on_prev_car |      3 |
+----+--------------------+-------------------------+--------+
|  3 |                  5 | loan_length             |      4 |
+----+--------------------+-------------------------+--------+
|  4 |                  5 | car_value               |      5 |
+----+--------------------+-------------------------+--------+
|  5 |                  0 | y_pred                  |      6 |
+----+--------------------+-------------------------+--------+
|  6 |                  0 | size_of_downpayment     |  

In [ ]:
estimated_calc = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    metrics=['roc_auc', 'recall'],
    chunk_size=5000,
    problem_type='classification_binary',
)
estimated_calc.fit(reference_df)
estimated_perf_results = estimated_calc.estimate(analysis_full_df)
display(estimated_perf_results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                      roc_auc                           ...  \
                 end_date    period     value sampling_error  realized  ...   
0 2018-11-30 00:27:16.848  analysis  0.968631       0.001811  0.970962  ...   
1 2018-12-30 07:03:16.848  analysis  0.969044       0.001811  0.970248  ...   
2 2019-01-29 13:39:16.848  analysis  0.969444       0.001811  0.976282  ...   
3 2019-02-28 20:15:16.848  analysis  0.969047       0.001811  0.967721  ...   
4 2019-03-31 02:51:16.848  analysis  0.968873       0.001811  0.969886  ...   
5 2019-04-30 09:27:16.848  analysis  0.960478       0.001811  0.960050  ...   
6 2019-05-30 16:03:16.848  analysis  0.961134       0.001811  0.958530  ...   
7 2019-06-29 22:39:16.848  analysis  0.960536       0.001811  0.959041  ...   
8 2019-07-30 05:15:16.848  analysis  0.961869       0.001811  0.963094  ...   
9 2019-08-29 11:51:16.848  analysis  0.960537       0.001811  0.957556  ...   

                            recall                           \
  lower_threshold  alert     value sampling_error  realized   
0        0.963317  False  0.928723       0.005137  0.930394   
1        0.963317  False  0.925261       0.005137  0.923922   
2        0.963317  False  0.929317       0.005137  0.938246   
3        0.963317  False  0.929713       0.005137  0.925060   
4        0.963317  False  0.930604       0.005137  0.927577   
5        0.963317   True  0.883990       0.005137  0.905086   
6        0.963317   True  0.883528       0.005137  0.899010   
7        0.963317   True  0.885501       0.005137  0.901718   
8        0.963317   True  0.885978       0.005137  0.906124   
9        0.963317   True  0.889808       0.005137  0.905823   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  0.944133                  0.913313        0.941033   
1                  0.940671                  0.909851        0.941033   
2                  0.944727                  0.913907        0.941033   
3                  0.945123                  0.914303        0.941033   
4                  0.946014                  0.915194        0.941033   
5                  0.899400                  0.868580        0.941033   
6                  0.898938                  0.868118        0.941033   
7                  0.900911                  0.870091        0.941033   
8                  0.901388                  0.870568        0.941033   
9                  0.905218                  0.874398        0.941033   

                          
  lower_threshold  alert  
0          0.9171  False  
1          0.9171  False  
2          0.9171  False  
3          0.9171  False  
4          0.9171  False  
5          0.9171   True  
6          0.9171   True  
7          0.9171   True  
8          0.9171   True  
9          0.9171   True  

[10 rows x 23 columns]

In [ ]:
print(estimated_perf_results.filter(period='analysis').to_df().to_markdown(tablefmt="grid"))

+----+--------------------+----------------------------+----------------------------+--------------------------+---------------------------+----------------------------+-----------------------+------------------------+---------------------------------+---------------------------+--------------------------------------------+--------------------------------------------+----------------------------------+----------------------------------+------------------------+-----------------------+--------------------------------+--------------------------+-------------------------------------------+-------------------------------------------+---------------------------------+---------------------------------+-----------------------+
|    | ('chunk', 'key')   |   ('chunk', 'chunk_index') |   ('chunk', 'start_index') |   ('chunk', 'end_index') | ('chunk', 'start_date')   | ('chunk', 'end_date')      | ('chunk', 'period')   |   ('roc_auc', 'value') |   ('roc_auc', 'sampling_error') |   ('roc_auc', 're

In [ ]:
realized_calc = nml.PerformanceCalculator(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    problem_type='classification_binary',
    metrics=['roc_auc', 'recall',],
    chunk_size=5000)
realized_calc.fit(reference_df)
realized_perf_results = realized_calc.calculate(analysis_full_df)
display(realized_perf_results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                                                roc_auc  \
                 end_date    period targets_missing_rate sampling_error   
0 2018-11-30 00:27:16.848  analysis                  0.0       0.001811   
1 2018-12-30 07:03:16.848  analysis                  0.0       0.001811   
2 2019-01-29 13:39:16.848  analysis                  0.0       0.001811   
3 2019-02-28 20:15:16.848  analysis                  0.0       0.001811   
4 2019-03-31 02:51:16.848  analysis                  0.0       0.001811   
5 2019-04-30 09:27:16.848  analysis                  0.0       0.001811   
6 2019-05-30 16:03:16.848  analysis                  0.0       0.001811   
7 2019-06-29 22:39:16.848  analysis                  0.0       0.001811   
8 2019-07-30 05:15:16.848  analysis                  0.0       0.001811   
9 2019-08-29 11:51:16.848  analysis                  0.0       0.001811   

                                                           recall            \
      value upper_threshold lower_threshold  alert sampling_error     value   
0  0.970962         0.97866        0.963317  False       0.005137  0.930394   
1  0.970248         0.97866        0.963317  False       0.005137  0.923922   
2  0.976282         0.97866        0.963317  False       0.005137  0.938246   
3  0.967721         0.97866        0.963317  False       0.005137  0.925060   
4  0.969886         0.97866        0.963317  False       0.005137  0.927577   
5  0.960050         0.97866        0.963317   True       0.005137  0.905086   
6  0.958530         0.97866        0.963317   True       0.005137  0.899010   
7  0.959041         0.97866        0.963317   True       0.005137  0.901718   
8  0.963094         0.97866        0.963317   True       0.005137  0.906124   
9  0.957556         0.97866        0.963317   True       0.005137  0.905823   

                                          
  upper_threshold lower_threshold  alert  
0        0.941033          0.9171  False  
1        0.941033          0.9171  False  
2        0.941033          0.9171  False  
3        0.941033          0.9171  False  
4        0.941033          0.9171  False  
5        0.941033          0.9171   True  
6        0.941033          0.9171   True  
7        0.941033          0.9171   True  
8        0.941033          0.9171   True  
9        0.941033          0.9171   True

In [ ]:
print(realized_perf_results.filter(period='analysis').to_df().to_markdown(tablefmt="grid"))

+----+--------------------+----------------------------+----------------------------+--------------------------+---------------------------+----------------------------+-----------------------+-------------------------------------+---------------------------------+------------------------+----------------------------------+----------------------------------+------------------------+--------------------------------+-----------------------+---------------------------------+---------------------------------+-----------------------+
|    | ('chunk', 'key')   |   ('chunk', 'chunk_index') |   ('chunk', 'start_index') |   ('chunk', 'end_index') | ('chunk', 'start_date')   | ('chunk', 'end_date')      | ('chunk', 'period')   |   ('chunk', 'targets_missing_rate') |   ('roc_auc', 'sampling_error') |   ('roc_auc', 'value') |   ('roc_auc', 'upper_threshold') |   ('roc_auc', 'lower_threshold') | ('roc_auc', 'alert')   |   ('recall', 'sampling_error') |   ('recall', 'value') |   ('recall', 'upper_th

In [ ]:
ranker1 = nml.CorrelationRanker()

# ranker fits on one metric and reference period data only
ranker1.fit(
    estimated_perf_results.filter(period='reference', metrics=['roc_auc']))
# ranker ranks on one drift method and one performance metric
correlation_ranked_features1 = ranker1.rank(
    univariate_results.filter(methods=['jensen_shannon']),
    estimated_perf_results.filter(metrics=['roc_auc']),
    only_drifting = False)

display(correlation_ranked_features1)

,column_name,pearsonr_correlation,pearsonr_pvalue,has_drifted,rank
0,repaid_loan_on_prev_car,0.998290,1.177706e-23,True,1
1,y_pred_proba,0.998072,3.474576e-23,True,2
2,loan_length,0.996876,2.661458e-21,True,3
3,salary_range,0.996512,7.162919e-21,True,4
4,car_value,0.996148,1.746760e-20,True,5
5,size_of_downpayment,0.307497,1.872200e-01,False,6
6,debt_to_income_ratio,0.250211,2.873424e-01,False,7
7,y_pred,0.075282,7.524257e-01,False,8
8,repaid,-0.117004,6.232446e-01,False,9
9,driver_tenure,-0.134447,5.719876e-01,False,10


In [ ]:
print(correlation_ranked_features1.to_markdown(tablefmt="grid"))

+----+-------------------------+------------------------+-------------------+---------------+--------+
|    | column_name             |   pearsonr_correlation |   pearsonr_pvalue | has_drifted   |   rank |
+====+=========================+========================+===================+===============+========+
|  0 | repaid_loan_on_prev_car |              0.99829   |       1.17771e-23 | True          |      1 |
+----+-------------------------+------------------------+-------------------+---------------+--------+
|  1 | y_pred_proba            |              0.998072  |       3.47458e-23 | True          |      2 |
+----+-------------------------+------------------------+-------------------+---------------+--------+
|  2 | loan_length             |              0.996876  |       2.66146e-21 | True          |      3 |
+----+-------------------------+------------------------+-------------------+---------------+--------+
|  3 | salary_range            |              0.996512  |       7.16292e-

In [ ]:
ranker2 = nml.CorrelationRanker()

# ranker fits on one metric and reference period data only
ranker2.fit(
    realized_perf_results.filter(period='reference', metrics=['recall']))
# ranker ranks on one drift method and one performance metric
correlation_ranked_features2 = ranker2.rank(
    univariate_results.filter(period='analysis', methods=['jensen_shannon']),
    realized_perf_results.filter(period='analysis', metrics=['recall']),
    only_drifting = False)

display(correlation_ranked_features2)

,column_name,pearsonr_correlation,pearsonr_pvalue,has_drifted,rank
0,repaid_loan_on_prev_car,0.968970,0.000004,True,1
1,y_pred_proba,0.966157,0.000006,True,2
2,loan_length,0.965298,0.000006,True,3
3,car_value,0.963623,0.000007,True,4
4,salary_range,0.963456,0.000007,True,5
5,size_of_downpayment,0.308948,0.385072,False,6
6,debt_to_income_ratio,0.307373,0.387627,False,7
7,y_pred,-0.357571,0.310383,False,8
8,repaid,-0.395842,0.257495,False,9
9,driver_tenure,-0.575807,0.081520,False,10


In [ ]:
print(correlation_ranked_features2.to_markdown(tablefmt="grid"))

+----+-------------------------+------------------------+-------------------+---------------+--------+
|    | column_name             |   pearsonr_correlation |   pearsonr_pvalue | has_drifted   |   rank |
+====+=========================+========================+===================+===============+========+
|  0 | repaid_loan_on_prev_car |               0.96897  |       3.90719e-06 | True          |      1 |
+----+-------------------------+------------------------+-------------------+---------------+--------+
|  1 | y_pred_proba            |               0.966157 |       5.50918e-06 | True          |      2 |
+----+-------------------------+------------------------+-------------------+---------------+--------+
|  2 | loan_length             |               0.965298 |       6.08385e-06 | True          |      3 |
+----+-------------------------+------------------------+-------------------+---------------+--------+
|  3 | car_value               |               0.963623 |       7.33185e-